<a href="https://colab.research.google.com/github/ipeirotis/autoencoders_census/blob/main/Autoencoder_YRBSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

dataset_url = "https://github.com/ipeirotis/autoencoders_census/raw/main/sadc_2017only_national_full.csv"
df = pd.read_csv(dataset_url)
print(df.head().to_markdown())

In [ ]:
columns_to_drop = ['sitecode', 'sitename', 'sitetype', 'sitetypenum', 'year', 'survyear', 'record']
df = df.drop(columns = columns_to_drop)
df1 = df.iloc[:, :100]

In [ ]:
df1.head()

In [ ]:
lst = [216, 232, 242, 245, 247, 249, 251, 252, 253, 256]
df1 = pd.concat([df1, df.iloc[:, lst]], axis=1)
df1.head()

In [ ]:
dict1 = {
        'age': 'age',
        'sex': 'sex', 'grade':'grade','race4':'Hispanic or Latino','race7':'race',
       'qnobese':'obese','qnowt':'overweight','q67':'sexual identity','q66':'sex/sexual contacts',
       'sexid':'sexid','sexid2':'sexid2','sexpart':'sexpart','sexpart2':'sexpart2','q8':'seat belt use','q9':'riding with a drinking driver',
       'q10':'drinking and driving','q11':'texting and driving','q12':'weapon carrying','q13':'weapon carrying at school',
       'q14':'gun carrying past 12 mos','q15':'safety concerns at school','q16':'threatened at school','q17':'physical fighting',
       'q18':'physical fighting at school','q19':'forced sexual intercourse','q20':'sexual violence','q21':'sexual dating violence',
       'q22':'physical dating violence','q23':'bullying at school','q24':'electronic bullying','q25':'sad or hopeless',
       'q26':'considered suicide','q27':'made a suicide plan','q28':'attempted suicide','q29':'injurious suicide attempt',
       'q30':'ever cigarette use','q31':'initation of cigarette smoking','q32':'current cigarette use','q33':'smoking amounts per day',
       'q34':'electronic vapor product use','q35':'current electronic vapor product use','q36':'EVP from store','q37':'current smokeless tobacco use',
       'q38':'current cigar use','q39':'all tobacco product cessation','q40':'ever alcohol use','q41':'initiation of alcohol use',
       'q42':'current alcohol use','q43':'source of alcohol','q44':'current binge drinking','q45':'largest number of drinks',
       'q46':'ever marijuana use','q47':'initiation of marijuana use','q48':'current marijuana use','q49':'ever cocaine use',
       'q50':'ever inhalant use','q51':'ever heroin use','q52':'ever methamphetamine use','q53':'ever ecstasy use',
       'q54':'ever synthetic marijuana use','q55':'ever steroid use','q56':'ever prescription pain medicine use','q57':'illegal injected drug use',
       'q58':'illegal drugs at school','q59':'ever sexual intercourse','q60':'first sex intercourse','q61':'multiple sex partners',
       'q62':'current sexual activity','q63':'alcohol/drugs at sex','q64':'condom use','q65':'birth control pill use',
       'q68':'perception of weight','q69':'weight loss','q70':'fruit juice drinking','q71':'fruit eating','q72':'green salad eating',
       'q73':'potato eating','q74':'carrot eating','q75':'other vegetable eating','q76':'soda drinking','q77':'milk drinking',
       'q78':'breakfast eating','q79':'physical activity','q80':'television watching','q81':'computer not school work use',
        'q82':'PE attendance','q83':'sports team participation','q84':'concussion in last 12 mos','q85':'HIV testing','q86':'oral health care',
       'q87':'asthma','q88':'sleep on school night','q89':'grades in school', 'qdrivemarijuana':'drive when using marijuana',
       'qhallucdrug':'ever used LSD', 'qsportsdrink':'sports drinks','qwater':'plain water','qfoodallergy':'food allergies',
        'qmusclestrength':'muscle stregthening','qindoortanning':'indoor tanning','qsunburn':'sunburn','qconcentrating':'difficulty concentrating',
        'qspeakenglish':'how well speak English'}

In [ ]:
df1.rename(columns=dict1, inplace = True)

In [ ]:
df1.isnull().sum()

In [ ]:
missing_percentages = df1.isnull().mean() * 100
columns_with_missing_gt_25 = missing_percentages[missing_percentages > 25].index

# Select the columns with missing values > 25%
selected_columns = df1[columns_with_missing_gt_25]

# Print the selected columns
print(selected_columns)

In [ ]:
# Iterate over each variable in the dataframe
for column in selected_columns:
  # create a new column name by appending
  missing_dummy_column = f'{column}_missing_dummy'
  df1[missing_dummy_column] = df1[column].isnull().astype(int)

# Display the updated dataframe
df1.head()

In [ ]:
data_num = df1[["weight","stratum","PSU","stheight","stweight","bmi","bmipct","obese","overweight"]]

In [ ]:
data_cat = df1.copy()
data_cat = data_cat.drop(data_num.columns, axis = 1)
data_cat.head()

In [ ]:
#scale on numerical columns--brings the value of each feature into the range of 0 to 1
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()
def scaleNum(df_num, cols):
    for col in cols:
        df_num[col] = pd.DataFrame(min_max_scaler.fit_transform(pd.DataFrame(data_num[col])),columns=[col])
    return df_num
data_normal_num = scaleNum(data_num,data_num.columns)

In [ ]:
data_normal_num.head()

In [ ]:
# label encoding on categorical columns
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
def scaleCat(df_cat, cols):
    for col in cols:
        df_cat[col] = pd.DataFrame(label_encoder.fit_transform(pd.DataFrame(data_cat[col])),columns=[col])
    return df_cat
data_normal_cat = scaleCat(data_cat,data_cat.columns)

In [ ]:
data_normal_cat.head()

In [ ]:
# merge numerical columns and categorical columns based on their indices
data = data_normal_num.merge(data_normal_cat, left_index = True, right_index = True)

In [ ]:
data.shape

In [ ]:
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras import backend as K


# the dimensionality of a latent space in an autoencoder
latent_dimension = 1
batch_size = 20
# the number of nuerons in a hidden layer
hidden_nodes = 16

# create the input layer for the encoder
input_encoder = Input(shape=(123,), name="Input_Encoder")
# apply batch normalization to the encoder input layer
batch_normalize1 = BatchNormalization()(input_encoder)
# create a hidden layer in the encoder
hidden_layer = Dense(hidden_nodes, activation="relu", name="Hidden_Encoding")(
    batch_normalize1
)
batch_normalize2 = BatchNormalization()(hidden_layer)
# create the output layer of the encoder
z = Dense(latent_dimension, name="Mean")(batch_normalize2)

In [ ]:
from keras import Model


encoder = Model(input_encoder, z)

In [ ]:
# create an input layer of the decoder
input_decoder = Input(shape=(latent_dimension,), name="Input_Decoder")
batch_normalize1 = BatchNormalization()(input_decoder)
# create a hidden layer
decoder_hidden_layer = Dense(hidden_nodes, activation="relu", name="Hidden_Decoding")(
    batch_normalize1
)
batch_normalize2 = BatchNormalization()(decoder_hidden_layer)
# create the output layer of the autoencoder
decoded = Dense(123, activation="linear", name="Decoded")(batch_normalize2)

In [ ]:
# specify the input and output layer of the decoder
decoder = Model(input_decoder, decoded, name="Decoder")

In [ ]:
# create a complete autoencoder architecture
encoder_decoder = decoder(encoder(input_encoder))

ae = Model(input_encoder, encoder_decoder)
# a summary of the autoencoder model
ae.summary()

In [ ]:
data.fillna(-1, inplace=True)

In [ ]:
from tensorflow.random import set_seed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# set the seed for random number generation
set_seed(2021)
# compile the complete autoencoder model
ae.compile(loss="mean_squared_error", optimizer="adam")

# use checkpoint during model training
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=1, verbose=0)

# train the autoencoder model on the input data
history = ae.fit(
    data, data, shuffle=True, epochs=10, batch_size=20, validation_split=0.2, verbose=0
).history

In [ ]:
from tensorflow.keras.models import load_model

ae = load_model('model.h5')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


sns.set(font_scale=2)
sns.set_style("white")


def model_analysis(history):
    # extract the training loss and validation loss values from the history object
    train_loss = history["loss"]
    val_loss = history["val_loss"]
    # x-axis values for the plot
    t = np.linspace(1, len(train_loss), len(train_loss))

    plt.figure(figsize=(16, 12))
    plt.title("Mean squared error")
    # plot the training loss and validation loss against the epoch values on the x-axis
    sns.lineplot(x=t, y=train_loss, label="Train", linewidth=3)
    sns.lineplot(x=t, y=val_loss, label="Validation", linewidth=3)
    plt.xlabel("Epochs")

    plt.legend()
    plt.savefig("FirstNet.png", dpi=400)
    plt.show()
    # the square root of the final training loss and validation loss values
    print(f"Training MSE = {np.sqrt(train_loss[-1])}")
    print(f"Validation MSE = {np.sqrt(val_loss[-1])}")


model_analysis(history)

In [ ]:
plt.figure(figsize=(12, 8))
plt.title("Empirical distribution function z")
plt.xticks((-8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14))
# use the encoder model to obtain the latent representation (z) of the data input
plt.hist(encoder.predict(data), bins=30, density=True)
plt.savefig("DistInternal.png", dpi=400)

In [ ]:
ae.predict(data)[0,:]

In [ ]:
# plot the empirical distribution function for the values obtained from the encoder's predictions on the dataset
from statsmodels.distributions.empirical_distribution import ECDF


ecdf = ECDF(encoder.predict(data)[:, 0])
plt.figure(figsize=(16, 12))
plt.title("Empirical distribution function z")
x = (-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5)
plt.yticks(np.linspace(0, 1, 11))
plt.xticks(x)
plt.grid()
plt.plot(x, ecdf(x), linewidth=3)
plt.savefig("EmpiricalDF.png", dpi=400)

In [ ]:
from scipy.interpolate import interp1d


x = (-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5)
# calculate the sample EDF values at the specified x-values
sample_edf_values_at_slope_changes = [ecdf(i) for i in x]
inverted_edf = interp1d(sample_edf_values_at_slope_changes, x)

In [ ]:
from numpy.random import uniform
from numpy.random import seed

# number of data points to generate
N = 10000
seed(2021)
plt.figure(figsize=(16, 12))
plt.title("Inverted empirical distribution function")
x = np.linspace(0.30, 0.80, 80)
plt.xticks(np.linspace(0, 1.0, 11))
plt.grid()
plt.plot(x, inverted_edf(x), linewidth=3)
plt.savefig("InvertedEmpiricalDF.png", dpi=400)

In [ ]:
# number of data points to generate
N = 10000
seed(2021)
plt.figure(figsize=(16, 12))
plt.title("Empirical distribution function z")
plt.xticks((-5, -4, -3, -2, -1, 0, 1, 2, 3, 4))
# N random values from a uniform distribution are transformed to follow a specific distribution using the inverted EDF
plt.hist(inverted_edf(uniform(0.30, 0.80, N)), bins=30, density=True)
plt.savefig("DistGenerated.png", dpi=400)

In [ ]:
import tensorflow as tf

plt.figure(figsize=(20, 8))
# reconstruct the original data
normal_reconstructions = ae.predict(data)
# compute the Mean Absolute Error between the reconstructed data and the original data
normal_loss = tf.losses.mae(normal_reconstructions, data)
plt.hist(normal_loss, bins=10)
plt.show()

In [ ]:
threshold = np.mean(normal_loss) + 2*np.std(normal_loss)
print(threshold)

In [ ]:
plt.figure(figsize=(20, 8))
plt.hist(normal_loss, bins=10, color='b', label="normal loss")
# add a vertical line to the plot at the position of the threshold value
plt.axvline(threshold, color='r', label="threshold")
plt.legend()
plt.show()

In [ ]:
# Get the indices that would sort the mae array in descending order
sorted_indices = np.argsort(normal_loss)[::-1]

In [ ]:
k = 100  # Number of samples to select
samples_with_high_error = data.iloc[sorted_indices[:k]]
samples_with_high_error.head()

In [ ]:
# Find indices of tuples with reconstruction values larger than the threshold
anomaly_indices = np.where(normal_loss > threshold)[0]

# Select the corresponding tuples from the original data
anomaly_tuples = data.iloc[anomaly_indices]

# Print the anomaly tuples
anomaly_tuples.head()


In [ ]:
anomaly_tuples.shape